# Example of running PhaseLink

<a href="https://colab.research.google.com/github/TomSHudson/PhaseLink/blob/master/example/example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Note that you need to change the run instance to GPU if using colab.

In [1]:
# Specify if running in google colab:
use_google_colab = False

# Install/add neccessary paths if using colab:
if use_google_colab:
    !pip install obspy
    # Install nvidia-apex:
    !git clone https://github.com/NVIDIA/apex
    !pip install -v --disable-pip-version-check --no-cache-dir --global-option="--cpp_ext" --global-option="--cuda_ext" ./apex
    

In [9]:
# Import neccessary modules:
import sys, os, shutil
import json
import multiprocessing as mp
import pickle
import numpy as np 
import torch
import gc 
import matplotlib.pyplot as plt
import glob
from obspy.geodetics.base import gps2dist_azimuth
# if not use_google_colab:
%load_ext autoreload
%autoreload 2
# And import PhaseLink:
if use_google_colab:
  shutil.rmtree('./PhaseLink', ignore_errors=True)
  !git clone https://github.com/TomSHudson/PhaseLink.git
  sys.path.append('./PhaseLink/')
else:
  sys.path.append('..')
import phaselink_dataset
import phaselink_train
import phaselink_eval

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
# Copy over example files into pwd if using colab:
if use_google_colab:
  !cp PhaseLink/example/params.json .
  !cp PhaseLink/example/station_list.txt .
  !cp PhaseLink/example/tt.pg .
  !cp PhaseLink/example/tt.sg .

## 0. Load in param file and key info:

In [11]:
# Import param json file:
params_fname = "params.json"
with open(params_fname, "r") as f:
    params = json.load(f)
    
# Get GPU info if using colab:
if use_google_colab:
     print("GPU:", torch.cuda.get_device_name(0))
     params['device'] = "cuda:0" # Use first GPU

## 1. Create a synthetic training dataset:

In [31]:
# Set key parameters from param file:
max_picks = params['n_max_picks']
t_max = params['t_win']
n_threads = params['n_threads']

print("Starting up...")
# Setup grid:
phase_idx = {'P': 0, 'S': 1}
lat0, lon0 = phaselink_dataset.get_network_centroid(params)
stlo, stla, phasemap, sncl_idx, stations, sncl_map = \
    phaselink_dataset.build_station_map(params, lat0, lon0, phase_idx)
x_min = np.min(stlo)
x_max = np.max(stlo)
y_min = np.min(stla)
y_max = np.max(stla)
for key in sncl_map:
    X0, Y0 = stations[key]
    X0 = (X0 - x_min) / (x_max - x_min)
    Y0 = (Y0 - y_min) / (y_max - y_min)
    stations[key] = (X0, Y0)

# Save station maps for detect mode
pickle.dump(stations, open(params['station_map_file'], 'wb'))
pickle.dump(sncl_map, open(params['sncl_map_file'], 'wb'))

# # Pwaves
# pTT = tt_interp(params['tt_table']['P'], params['datum'])
# print('Read pTT')
# print('(dep,dist) = (0,0), (10,0), (0,10), (10,0):')
# print('             {:.3f}   {:.3f}   {:.3f}   {:.3f}'.format(
# pTT.interp(0,0), pTT.interp(10,0),pTT.interp(0,10),
# pTT.interp(10,10)))

# #Swaves
# sTT = tt_interp(params['tt_table']['S'], params['datum'])
# print('Read sTT')
# print('(dep,dist) = (0,0), (10,0), (0,10), (10,0):')
# print('             {:.3f}   {:.3f}   {:.3f}   {:.3f}'.format(
# sTT.interp(0,0), sTT.interp(10,0),sTT.interp(0,10),
# sTT.interp(10,10)))

# Get travel-time tables for P and S waves:
pTT = phaselink_dataset.tt_interp(params['trav_time_p'], params['datum'])
sTT = phaselink_dataset.tt_interp(params['trav_time_s'], params['datum'])

# Generate synthetic training dataset for given param file:
in_q = mp.Queue() ###1000000)
out_q = mp.Queue() ###1000000)
proc = mp.Process(target=phaselink_dataset.output_thread, args=(out_q, params))
proc.start()
procs = []
for i in range(n_threads):
    print("Starting thread %d" % i)
    p = mp.Process(target=phaselink_dataset.generate_phases, \
        args=(in_q, out_q, x_min, x_max, y_min, y_max, \
              sncl_idx, stla, stlo, phasemap, pTT, sTT, params))
    p.start()
    procs.append(p)

for i in range(params['n_train_samp']):
    in_q.put(i)

for i in range(n_threads):
    in_q.put(None)
#for p in procs:
#    p.join()
#proc.join()

print("Creating the following files for the PhaseLink synthetic training dataset:")
print(params['station_map_file'])
print(params['sncl_map_file'])
print(params['training_dset_X'])
print(params['training_dset_Y'])

Starting up...
Starting thread 0
Starting thread 1
Starting thread 2
Starting thread 3
Starting thread 4
Starting thread 5
Starting thread 6
Starting thread 7
Creating the following files for the PhaseLink synthetic training dataset:
station_map.pkl
sncl_map.pkl
shimane_train_X.npy
shimane_train_Y.npy
P-phases (zeros): 11973180 ( 98.0604422604 %)
S-phases (ones): 236820 ( 1.93955773956 %)
Saved the synthetic training dataset.


## 2. Train the model using the syntehetic dataset:

In [32]:
# Get device (cpu vs gpu) specified:
device = torch.device(params["device"])
if params["device"][0:4] == "cuda":
    torch.cuda.empty_cache()
    enable_amp = True
else:
    enable_amp = False
if enable_amp:
    import apex.amp as amp

# Get training info from param file:
n_epochs = params["n_epochs"] #100

# Load in training dataset:
X = np.load(params["training_dset_X"])
Y = np.load(params["training_dset_Y"])
print("Training dataset info:")
print("Shape of X:", X.shape, "Shape of Y", Y.shape)
dataset = phaselink_train.MyDataset(X, Y, device)

# Get dataset info:
n_samples = len(dataset)
indices = list(range(n_samples))

# Set size of training and validation subset:
n_test = int(0.1*X.shape[0])
validation_idx = np.random.choice(indices, size=n_test, replace=False)
train_idx = list(set(indices) - set(validation_idx))

# Specify samplers:
train_sampler = phaselink_train.SubsetRandomSampler(train_idx)
validation_sampler = phaselink_train.SubsetRandomSampler(validation_idx)

# Load training data:
train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=256,
    shuffle=False,
    sampler=train_sampler
)
val_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=1024,
    shuffle=False,
    sampler=validation_sampler
)

stackedgru = phaselink_train.StackedGRU()
stackedgru = stackedgru.to(device)
#stackedgru = torch.nn.DataParallel(stackedgru,
#    device_ids=['cuda:2', 'cuda:3', 'cuda:4', 'cuda:5'])

if enable_amp:
    #amp.register_float_function(torch, 'sigmoid')
    from apex.optimizers import FusedAdam
    optimizer = FusedAdam(stackedgru.parameters())
    stackedgru, optimizer = amp.initialize(
        stackedgru, optimizer, opt_level='O2')
else:
    optimizer = torch.optim.Adam(stackedgru.parameters())

model = phaselink_train.Model(stackedgru, optimizer, \
    model_path='./phaselink_model')
print("Begin training process.")
model.train(train_loader, val_loader, n_epochs, enable_amp=enable_amp)


Training dataset info:
Shape of X: (8140, 1500, 5) Shape of Y (8140, 1500)
Begin training process.
Epoch 1, 10% 	 train_loss: 8.1183e-01 train_acc: 98.24% took: 38.19s
Epoch 1, 20% 	 train_loss: 4.4219e-01 train_acc: 98.07% took: 35.93s
Epoch 1, 31% 	 train_loss: 1.7653e-01 train_acc: 98.08% took: 35.90s
Epoch 1, 41% 	 train_loss: 1.5708e-01 train_acc: 98.06% took: 37.58s
Epoch 1, 51% 	 train_loss: 1.7691e-01 train_acc: 98.07% took: 36.36s
Epoch 1, 62% 	 train_loss: 1.9457e-01 train_acc: 98.06% took: 35.61s
Epoch 1, 72% 	 train_loss: 1.6042e-01 train_acc: 98.08% took: 35.39s
Epoch 1, 82% 	 train_loss: 1.6453e-01 train_acc: 98.07% took: 35.34s
Epoch 1, 93% 	 train_loss: 1.4670e-01 train_acc: 98.06% took: 36.86s
Precision (Class 0): 0.980%
Recall (Class 0): 1.000%


ZeroDivisionError: division by zero

In [33]:
# For emptying memory on GPU:
# torch.cuda.empty_cache()
# del(model)
# gc.collect()
# torch.cuda.empty_cache()

In [ ]:
# Download trained model, if using colab:
if use_google_colab:
    from google.colab import files
    !zip -r ./phaselink_model/phaselink_model.zip ./phaselink_model
    files.download('phaselink_model/phaselink_model.zip') 

In [59]:
# Plot model training and validation loss to select best model:
# (Note: This must currently be done on the same machine/machine architecture as 
#  the training was undertaken on).

# Write the models loss function values to file:
models_dir = "phaselink_model"
models_fnames = list(glob.glob(os.path.join(models_dir, "model_???_*.pt")))
models_fnames.sort()
val_losses = []
f_out = open(os.path.join(models_dir, 'val_losses.txt'), 'w')
for model_fname in models_fnames:
    model_curr = torch.load(model_fname)
    val_losses.append(model_curr['loss'])
    f_out.write(' '.join((model_fname, str(model_curr['loss']), '\n')))
    del(model_curr)
    gc.collect()
f_out.close()
val_losses = np.array(val_losses)
print("Written losses to file: ", os.path.join(models_dir, 'val_losses.txt'))

# And select approximate best model (approx corner of loss curve):
approx_corner_idx = np.argwhere(val_losses < np.average(val_losses))[0][0]
print("Model to use:", models_fnames[approx_corner_idx])

# And plot:
plt.figure()
plt.plot(np.arange(len(val_losses)), val_losses)
plt.hlines(val_losses[approx_corner_idx], 0, len(val_losses), color='r', ls="--")
plt.ylabel("Val loss")
plt.xlabel("Epoch")
plt.show()

# And convert model to use to universally usable format (GPU or CPU):
model = phaselink_train.StackedGRU().cuda(device)
checkpoint = torch.load(models_fnames[approx_corner_idx], map_location=device)
model.load_state_dict(checkpoint['model_state_dict'])
torch.save(model, os.path.join(models_dir, 'model_to_use.gpu.pt'), _use_new_zipfile_serialization=False)
new_device = "cpu"
model = model.to(new_device)
torch.save(model, os.path.join(models_dir, 'model_to_use.cpu.pt'), _use_new_zipfile_serialization=False)
del model
gc.collect()
if use_google_colab:
  files.download(os.path.join('phaselink_model','model_to_use.gpu.pt'))
  files.download(os.path.join('phaselink_model','model_to_use.cpu.pt'))


RuntimeError: version_ <= kMaxSupportedFileFormatVersion INTERNAL ASSERT FAILED at /Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/caffe2/serialize/inline_container.cc:132, please report a bug to PyTorch. Attempted to read a PyTorch file with version 3, but the maximum supported version for reading is 2. Your PyTorch installation may be too old. (init at /Users/distiller/project/conda/conda-bld/pytorch_1579022061893/work/caffe2/serialize/inline_container.cc:132)
frame #0: c10::Error::Error(c10::SourceLocation, std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&) + 135 (0x19d6b6267 in libc10.dylib)
frame #1: caffe2::serialize::PyTorchStreamReader::init() + 2350 (0x19858650e in libtorch.dylib)
frame #2: caffe2::serialize::PyTorchStreamReader::PyTorchStreamReader(std::__1::basic_string<char, std::__1::char_traits<char>, std::__1::allocator<char> > const&) + 143 (0x198585b5f in libtorch.dylib)
frame #3: void pybind11::cpp_functio

## 3. Perform phase association on some real earthquakes:

In [7]:
# Load correct model:
if use_google_colab:
    params["model_file"] = "phaselink_model/model_to_use.gpu.pt"
    model = torch.load(params["model_file"])
    
else:
    params["model_file"] = "phaselink_model/model_to_use.cpu.pt"
    model = torch.load(params["model_file"])

# And evaluate model:
model.eval()

/Users/eart0504/opt/anaconda3/envs/phaselink/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
/Users/eart0504/opt/anaconda3/envs/phaselink/lib/python3.7/site-packages/torch/serialization.py:593: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.GRU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


StackedGRU(
  (fc1): Linear(in_features=5, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=32, bias=True)
  (fc5): Linear(in_features=32, out_features=32, bias=True)
  (fc6): Linear(in_features=256, out_features=1, bias=True)
  (gru1): GRU(32, 128, batch_first=True, bidirectional=True)
  (gru2): GRU(256, 128, batch_first=True, bidirectional=True)
)

In [15]:
# Detect events:
X, labels = phaselink_eval.read_gpd_output(params)
phaselink_eval.detect_events(X, labels, model, params)

print("Events output to .nlloc file.")

Reading GPD file
Finished reading GPD file, 100000 triggers found
Day 001: 67693 gpd picks, 0 cumulative events detected
Permuting sequence for all lags...
Finished permuting sequence
Predicting labels for all phases
Finished label prediction
Linking phases
20 events detected initially
Removing duplicates
13 events detected after duplicate removal
13 events left after applying n_min_det threshold
Day 002: 32307 gpd picks, 13 cumulative events detected
Permuting sequence for all lags...
Finished permuting sequence
Predicting labels for all phases
Finished label prediction
Linking phases
36 events detected initially
Removing duplicates
35 events detected after duplicate removal
35 events left after applying n_min_det threshold
48 detections total
Events output to .nlloc file.
